# PART 2: Dive Into RobotFramework

----

## RobotFramework的开发模式探寻

我们首先来探寻一下RobotFramework的开发模式. 通过观察它在github上的项目以及提交和发布情况,我们可以发现:
* 开发基于[github](https://github.com/robotframework/robotframework)
* 工作在master分支上
* 不直接提交到master分支
* 使用**pull request**来进行code review
* 使用milestone来进行项目进度管理,如https://github.com/robotframework/robotframework/milestones
* 使用branch来进行feature开发
* 使用tag来进行release

----

## RobotFramework的测试

### Unit Testing

```sh
python utest/run_utests.py
```

### Acceptance Testing

```sh
python atest/run_atests.py python robot
```

----

## RobotFramework的整体工作流

![RobotFramework Workflow](img/robot_workflow.svg "RobotFramework Workflow")


----

## RobotFramework的主要模块

In [1]:
import robot
filter(lambda x: not x.startswith('_') and x not in ('sys', 'version', 'get_version', 'pythonpathsetter', 'run_cli', 'rebot', 'rebot_cli'), dir(robot))

['conf',
 'errors',
 'htmldata',
 'model',
 'output',
 'parsing',
 'reporting',
 'result',
 'run',
 'running',
 'utils',
 'variables',
 'writer']

我们考虑其中主要的模块:
* robot.run
* robot.model
* robot.parsing
* robot.running
* robot.output
* robot.result
* robot.reporting


----

### robot.run

In [15]:
from robot.run import RobotFramework # main class
from robot.run import run # 执行入口
from robot.run import run_cli # 命令行执行入口

RobotFramework的主要代码
```python
class RobotFramework(Application):

    def __init__(self):
        Application.__init__(self, USAGE, arg_limits=(1,),
                             env_options='ROBOT_OPTIONS', logger=LOGGER)

    def main(self, datasources, **options):
        settings = RobotSettings(options)
        LOGGER.register_console_logger(**settings.console_logger_config)
        LOGGER.info('Settings:\n%s' % unicode(settings))
        suite = TestSuiteBuilder(settings['SuiteNames'],
                                 settings['WarnOnSkipped'],
                                 settings['RunEmptySuite']).build(*datasources)
        suite.configure(**settings.suite_config)
        with pyloggingconf.robot_handler_enabled(settings.log_level):
            result = suite.run(settings)
            LOGGER.info("Tests execution ended. Statistics:\n%s"
                        % result.suite.stat_message)
            if settings.log or settings.report or settings.xunit:
                writer = ResultWriter(settings.output if settings.log
                                      else result)
                writer.write_results(settings.get_rebot_settings())
        return result.return_code
```

run和run_cli的代码
```python
def run_cli(arguments):
    RobotFramework().execute_cli(arguments)


def run(*datasources, **options):
    return RobotFramework().execute(*datasources, **options)
```

----

### robot.model

类似通常**MVC**框架中的model, 是RobotFramework中的数据抽象层.
robot.model中的代码大致包含以下几类:

* 基本类型
* RobotFramework  suite/case/keyword等的抽象
* 构建上述抽象类的工厂
* 对抽象类的装饰(如filter)
* visitor

In [23]:
# 基本类型
from robot.model.modelobject import ModelObject
from robot.model.itemlist import ItemList

In [28]:
# RobotFramework suite/case/keyword等的抽象
from robot.model.testsuite import TestSuite, TestSuites
from robot.model.testcase import TestCase, TestCases
from robot.model.imports import Import, Imports
from robot.model.keyword import Keyword, Keywords
from robot.model.message import Message, Messages
from robot.model.metadata import Metadata
from robot.model.tags import Tags
from robot.model.suitestatistics import SuiteStatistics
from robot.model.tagstatistics import TagStatistics
from robot.model.totalstatistics import TotalStatistics
from robot.model.stats import SuiteStat, TagStat, TotalStat

In [29]:
# 构建抽象类的工厂
from robot.model.configurer import SuiteConfigurer
from robot.model.statistics import  StatisticsBuilder
from robot.model.suitestatistics import  SuiteStatisticsBuilder
from robot.model.tagstatistics import  TagStatisticsBuilder
from robot.model.totalstatistics import TotalStatisticsBuilder

In [30]:
# 对抽象类的装饰
from robot.model.tags import TagPattern, TagPatterns # NOT, AND, OR
from robot.model.criticality import Criticality

In [ ]:
# visitor
from robot.model.visitor import SuiteVisitor, SkipAllVisitor
from robot.model.tagsetter import  TagSetter
from robot.model.filter import  Filter, EmptySuiteRemover

----

### robot.parsing

顾名思义, robot.parsing是RobotFramework的解析模块,负责将suite文件或目录解析成RobotFramework自己的数据结构. 

In [31]:
# case文件或目录的解析模块
from robot.parsing.htmlreader import  HtmlReader
from robot.parsing.restreader import RestReader
from robot.parsing.txtreader import TxtReader
from robot.parsing.tsvreader import TsvReader

最主要的接口 - TestData

In [32]:
# case文件或目录到model的转换 - robot.parsing.model.TestData
from robot.parsing.model import  TestData
from robot.api import TestData # 也可以直接用api调用, 这样更好

In [33]:
# 行解析robot.parsing.datarow
from robot.parsing.datarow import DataRow

In [ ]:
# settings_table的数据结构robot.parsing.settings
from robot.parsing.settings import Documentation, Library, Metadata, Resource # ...

----

### robot.running

robot.running是RobotFramework运行时的主要模块, RobotFramework的整个运行阶段基本都与它相关

#### TestSuiteBuilder - 创建运行时的TestSuite

In [40]:
from robot.running.builder import TestSuiteBuilder
from robot.api import TestSuiteBuilder # 也可以直接通过api调用,这样更好
suite = TestSuiteBuilder().build('examples/kw-driven.robot')
print suite

Kw-Driven


#### EXECUTION_CONTEXTS - 运行时的所有信息

In [2]:
# coding: utf-8
# example - 在当前每个test运行前插入一个分割符的log
from robot.running.context import EXECUTION_CONTEXTS
from robot.running import Keyword

class TestSplitter:
    ROBOT_LISTENER_API_VERSION = 2

    def __init__(self):
        self._splitter = '-' * 80

    def start_test(self, name, attributes):
        ns = EXECUTION_CONTEXTS.current
        Keyword('log', (self._splitter, )).run(ns)


使用命令`pybot --listener listener.TestSplitter kw-driven.robot`执行，可以得到如下的log：

[log.html](examples/log.html)

#### model - 运行时的模型抽象

In [70]:
from robot.running.model import TestCase, TestSuite
from robot.running.keywords import Keyword, Keywords
from robot.running.status import SuiteStatus, TestStatus
from robot.running.userkeyword import UserLibrary # 还有其它模型抽象
from robot.running.testlibraries import TestLibrary

#### 基本类型

In [71]:
from robot.running.baselibrary import BaseLibrary
from robot.running.dynamicmethods import RunKeyword
from robot.running.handlers import Handler

#### 传递给某个具体的suite的运行数据

In [47]:
from robot.running.namespace import Namespace

#### Runner - 实际执行suite的class

In [ ]:
from robot.running.runner import Runner
# dir(Runner) # 可以看到如start_suite等方法

#### signal监控 - 处理stop gracefully等

In [4]:
from robot.running.signalhandler import STOP_SIGNAL_MONITOR
with STOP_SIGNAL_MONITOR:
    import time
    print 'a'
    time.sleep(2)
    print 'b'
    time.sleep(2)
    print 'c'
    time.sleep(2)

a
b
c


Question:

1. 为什么上面的例子还是把a,b,c都输出来了?
1. 为什么执行Ctrl+C后之后的keyword都不执行了?

#### timeout处理

原理：根据平台和python版本不同，timeout有几种不同的实现方式。
```python
if sys.platform == 'cli':
    from .timeoutthread import Timeout
elif os.name == 'nt':
    from .timeoutwin import Timeout
else:
    try:
        # python 2.6 or newer in *nix or mac
        from .timeoutsignaling import Timeout
    except ImportError:
        # python < 2.6 and jython don't have complete signal module
        from .timeoutthread import Timeout
```
以Linux平台Python 2.7为例来展示一下它的工作方式。

In [6]:
from robot.running.timeouts import Timeout
t = Timeout(1, 'timeout')

def test():
    import time
    print 'start'
    time.sleep(10)
    print 'end'

t.execute(test)

start


TimeoutError: timeout

In [10]:
# 它的内部工作原理还是基于signal
from signal import setitimer, signal, SIGALRM, ITIMER_REAL

def _raise_error(signum, frame):
    raise RuntimeError('timeout')
    
def test():
    import time
    print 'start'
    time.sleep(10)
    print 'end'
    
signal(SIGALRM, _raise_error)
setitimer(ITIMER_REAL, 1)
test()

start


RuntimeError: timeout

#### arguments处理

In [53]:
from robot.running.arguments import argumentparser # ...

#### defaults - 运行时的默认数据

In [69]:
from robot.running.defaults import TestDefaults # 基于robot.parsing.model.TestData得到的setting_table
from robot.running.defaults import TestValues # 基于robot.parsing.model.TestData得到的testcase_table
from robot.api import TestData
data = TestData(source='examples/kw-driven.robot')
defaults = TestDefaults(data.setting_table)
values = TestValues(data.testcase_table.tests[0], defaults)
#print values.template, values.timeout, values.tags

#### 某些工具类

In [72]:
from robot.running.importer import Importer
from robot.running.outputcapture import OutputCapturer
from robot.running.randomizer import Randomizer
from robot.running.runkwregister import RUN_KW_REGISTER
from robot.running.usererrorhandler import UserErrorHandler

----

### robot.result

**robot.result**是XML output的解析模块, 它的主要功能是将output.xml转换为Result对象

In [ ]:
# 从output.xml文件创建Result对象
from robot.result import  ExecutionResult
result = ExecutionResult('output.xml')

In [ ]:
#  Result vs. CombinedResult
from robot.result.resultbuilder import CombinedResult, Result, Merger

#### result阶段的model

In [ ]:
from robot.result.testsuite import TestSuite
from robot.result.testcase import  TestCase
from robot.result.keyword import Keyword

----

### robot.reporting

**robot.reporting**主要用于生成log.html和report.html, 除此以外, 它的功能还有:

* 生成新的output.xml
* log.html中js部分的生成器 (用于动态加载优化性能和解决内存问题)

In [73]:
# log.html & report.html
from robot.reporting.resultwriter import ResultWriter, ReportWriter

In [74]:
# output.xml
from robot.reporting.outputwriter import OutputWriter

In [75]:
# js writer
from robot.reporting.jswriter import JsResultWriter, SplitLogWriter, JsonWriter

----

## 不同阶段的model

在不同的阶段都有对应的model, 另外robot还有一个robot.model模块.

大部分的model都继承自robot.model中的相关类.

但是robot.parsing.model除外. 怀疑是RobotFramework的遗留代码没有重构完导致的.

* robot.model
* robot.parsing.model
* robot.running.model
* robot.result."model"

----

## Q: 你知道执行pybot后RobotFramework都做了些什么吗?

* 实例化robot.run.RobotFramework
* 命令行参数解析
* 实例化RobotSettings
* 设置LOGGER
* 生成TestSuite
* 执行TestSuite
* 写入log和report
* 判断测试结果
* 结束进程

这里写入log和report为optional步骤

In [82]:
from robot.run import run_cli
run_cli(['examples/kw-driven.robot'])

SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.


----

## Q: 你知道RobotFramework是怎么找到指定的keyword的吗?

1. 当前case中的user keyword
1. resource文件中定义的user keyword
1. library中定义的keyword

In [ ]:
# 相关代码
# robot.running.namespace
def _get_implicit_handler(self, name):
    for method in [self._get_handler_from_test_case_file_user_keywords,
                   self._get_handler_from_resource_file_user_keywords,
                   self._get_handler_from_library_keywords]:
        handler = method(name)
        if handler:
            return handler
    return None

----

## Q: 你知道RobotFramework是如何导入Library的?

1. 将library名字import为class或者module
1. 根据不同的library类型生成对应的Library class

In [83]:
# 相关代码
# robot.running.testlibraries
def TestLibrary(name, args=None, variables=None, create_handlers=True):
    with OutputCapturer(library_import=True):
        importer = Importer('test library')
        libcode = importer.import_class_or_module(name)
    libclass = _get_lib_class(libcode)
    lib = libclass(libcode, name, args or [], variables)
    if create_handlers:
        lib.create_handlers()
    return lib

In [85]:
# 引申: module library的keyword来源

# robot.running.testlibraries
# 动态library的keywords列表
def _get_handler_names(self, instance):
    try:
        return instance.get_keyword_names()
    except AttributeError:
        return instance.getKeywordNames()

# robot.running.namespace
# 处理keyword名字的方式
def _get_handler(self, name):
    handler = None
    if not name:
        raise DataError('Keyword name cannot be empty.')
    if not isinstance(name, basestring):
        raise DataError('Keyword name must be a string.')
    if '.' in name:
        handler = self._get_explicit_handler(name)
    if not handler:
        handler = self._get_implicit_handler(name)
    if not handler:
        handler = self._get_bdd_style_handler(name)
    if not handler:
        handler = self._get_x_times_handler(name)
    return handler

----

## Q: 你知道suite的默认执行顺序是什么吗?

![suite order](img/suite.png)

In [86]:
# 相关代码
# robot.parsing.populators.FromDirectoryPopulator
def _list_dir(self, path):
    # os.listdir returns Unicode entries when path is Unicode
    names = os.listdir(unic(path))
    for name in sorted(names, key=unicode.lower):
        # unic needed to handle nfc/nfd normalization on OSX
        yield unic(name), unic(os.path.join(path, name))

----

## Q: 你知道suite的随机执行顺序是什么样的吗?

In [90]:
from robot.model.visitor import SuiteVisitor
class Randomizer(SuiteVisitor):

    def __init__(self, randomize_suites=True, randomize_tests=True, seed=None):
        # ......
        args = (seed,) if seed is not None else ()
        self._shuffle = Random(*args).shuffle

    def start_suite(self, suite):
        # ......
        if self.randomize_suites:
            self._shuffle(suite.suites)
        if self.randomize_tests:
            self._shuffle(suite.tests)
        # ......
        
# 类似效果如下:
from random import Random
shuffle = Random().shuffle
_set = range(10)
shuffle(_set)
_set

[8, 4, 6, 9, 5, 0, 1, 3, 2, 7]

----

## Q: 你知道Listener是怎么工作的吗?

* robot.output.listeners
* 将listeners追加到LOGGER
* 进行LOGGER操作的时候执行listener的对应接口

----

## Q: 你知道RobotFramework的Stop Gracefully是怎么实现的吗?

In [4]:
# robot.running.model.TestSuite的run方法
# robot.running.signalhandler
# 通过signal.signal来实现Ctrl+C的监听
from robot.running.signalhandler import STOP_SIGNAL_MONITOR
from robot.output import pyloggingconf

# robot.running.model
def run(self, settings=None, **options):
    if not settings:
        settings = RobotSettings(options)
        LOGGER.register_console_logger(**settings.console_logger_config)
    with pyloggingconf.robot_handler_enabled(settings.log_level):
        with STOP_SIGNAL_MONITOR:
            IMPORTER.reset()
            init_global_variables(settings)
            output = Output(settings)
            runner = Runner(output, settings)
            self.visit(runner)
        output.close(runner.result)
    return runner.result


# robot.running.handlers
def _run_with_signal_monitoring(self, runner, context):
    try:
        STOP_SIGNAL_MONITOR.start_running_keyword(context.in_teardown)
        return runner()
    finally:
        STOP_SIGNAL_MONITOR.stop_running_keyword()


----

## Q: 你知道RobotFramework的Test Timeout是怎么做的吗?

* 执行时在开始执行test或keyword的时候, 设置Timeout
    * robot.running.model.TestCase
* 以Python2.7 Linux平台为例
    * robot.running.timeouts.timeoutsignaling.Timeout
* 通过signal模块的setitimer和SIGALRM信号来实现

In [95]:
from signal import signal, setitimer, SIGALRM, ITIMER_REAL
def show_error(signum, frame):
    print 'timeout got'
signal(SIGALRM, show_error)
setitimer(ITIMER_REAL, 5)

(0.0, 0.0)

----

## 练习一: 阅读相关代码, 理清RobotFramework是怎么写入log的

## 练习二: 阅读相关代码, 理清dry-run是怎么实现的,有哪些局限性